<a href="https://colab.research.google.com/github/ChandramouliSridharan/Mental-Health-Sentiment-Analysis-using-Deep-Learning./blob/main/Mental_Health_Sentiment_Analysis_ProjectGroup5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Importing necessary libraries**

In [ ]:
# ! pip install tensorfloW

In [ ]:
# Install this package Initially for the BERT model to run.
!pip install tensorflow==2.15.0 tensorflow-text==2.15.0 tensorflow-hub==0.16.1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import seaborn as sns

import re
import random
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Embedding, Dropout, Input, concatenate, MaxPooling1D, Conv1D
from keras.callbacks import ModelCheckpoint

from xgboost import XGBClassifier

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
nltk.download('punkt_tab')
nltk.download('stopwords')

**Reading data from a CSV file**.

In [ ]:
# Creating a datframe.
data = pd.read_csv("/content/Combined Data.csv", index_col = 0)
# Displaying 5 random rows from the DataFrame to inspect the data.
data.sample(5)

***Analyzing the data***

Checking the dimensions of the DataFrame (number of rows and columns).

In [ ]:
data.shape

Calculate the length of each statements.

In [ ]:
# Calculate the character length of each statement
data['char_count'] = data['statement'].str.len()

# Display basic statistics of statement lengths
print(data['char_count'].describe())
data.head(5)

***Visualizing the distribution of the lengths of statements.***

In [ ]:
data['char_count'].hist(bins=100, color='skyblue', range=(1,15000))
plt.title('Frequency Distribution of Statement Lengths')
plt.xlabel('Number of Characters in the Statements')
plt.ylabel('Statement Counts')
plt.show()

Filter the data for statements longer than 5000 characters

In [ ]:
filtered_df = data[data['char_count']>5000]
filtered_df.count()

Getting the count of statements in each category to understand how diverse the data is.

In [ ]:
data['status'].value_counts()

Visualize the count of statements in each category as a bar graph.

In [ ]:
category_count=data['status'].value_counts()
category_count.plot(kind='bar', title='Distribution of Sentiments of Different Categories',
color='#800000')

Creating a pie chart to visualize the distribution of the 'status' column

In [ ]:
fig = px.pie(data, names='status',labels='status', title='Mental Health Category Distribution')
fig.show()

**DATA CLEANING AND DATA PRE-PROCESSING**

Checking whether the null values are present in each column of the dataframe.

In [ ]:
data.isnull().sum()

Dropping rows with null values from the DataFrame

In [ ]:
# removing null values
data.dropna(inplace = True)
data.isna().sum()

Preprocessing the text data by converting statements to lowercase.

In [ ]:
data['statement']=data['statement'].str.lower()
data.sample(5)

Cleaning the data of the 'statement' column based on Regex pattern.

In [ ]:
def remove_expression(text):
    # Remove URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    # Remove markdown-style links
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)
    # Remove handles (that start with '@')
    text = re.sub(r'@\w+', '', text)
    # Remove punctuation and other special characters
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

# Apply the function to the 'statement' column
data['statement'] = data['statement'].apply(remove_expression)
data.sample(5)

Tokenizing the statements.

In [ ]:
# Apply word_tokenize to each element in the 'statement' column(tokenization)
data['tokens_list'] = data['statement'].apply(word_tokenize)
data.sample(5)

Removing stopwords and Stemming using Porter stemmer from the tokens

In [ ]:
# Stemming
# Initialize the stemmer
stopwords_list = stopwords.words('english')
stemmer = PorterStemmer()

# Function to stem tokens and convert them to strings
def stem_tokens_list(tokens_list):
    return ' '.join(stemmer.stem(str(token)) for token in tokens_list if token not in stopwords_list )

# Apply the function to the 'tokens' column
data['stemmed_tokens'] = data['tokens_list'].apply(stem_tokens_list)

data.sample(5)

Creating WordCloud to identify the important and repeated tokens.

In [ ]:
# Word Cloud
colors = ['#16325B', '#227B94', '#78B7D0', '#FFDC7F', '#18587A', '#11999E', '#283644']

# Define a color function that randomly selects colors from the palette
def color_function(word, font_size, position, orientation, random_state=101, **kwargs):
    return random.choice(colors)

Wordcloud for each mental health status category.

In [ ]:
# Get unique categories in 'status'
categories  = data['status'].unique()

plt.figure(figsize=(12, 36))

# Generate and plot the WordCloud for each category
for i, category  in enumerate(categories):

    # Filter the tokens data for the current status
    total_tokens = ' '.join(data[data['status'] == category]['tokens_list'].dropna().apply(lambda tokens: ' '.join(tokens)).tolist())

    # Generate the WordCloud
    wc = WordCloud(width=800, height=400, background_color='white', color_func=color_function).generate(total_tokens)

    # Plot the WordCloud in a subplot
    axes = plt.subplot(len(categories) // 2 + 1, 2, i + 1)
    plt.imshow(wc, interpolation='bilinear')
    plt.title(f'WordCloud for Status: {category}')

# Apply tight layout after generating all subplots
plt.tight_layout()

# Adjust the vertical spacing between subplots (hspace controls vertical space)
plt.subplots_adjust(hspace= -0.8)
plt.show()

In [ ]:
Lb = LabelEncoder()
data['status'] = Lb.fit_transform(data['status'])
print(data['status'].unique())

In [ ]:
row_counts = []

for i in range(7):  # For each status from 0 to 6
    subset = data[data['status'] == i]
    count = len(subset)
    row_counts.append(count)  # Store the count
    print(f"Status {i}: {count} rows")

# Find the maximum value
max_rows = max(row_counts)
print(f"The maximum number of rows is: {max_rows}")


In [ ]:
from sklearn.utils import resample

# Desired sample size for each group (e.g., equal-sized groups for balancing)
desired_sample_size = max_rows  # Adjust based on your needs

resampled_dfs = []

for i in range(7):  # Status values from 0 to 6
    subset = data[data['status'] == i]

    # Resample each group
    resampled_subset = resample(
        subset,
        replace=True if len(subset) < desired_sample_size else False,  # Use replacement if the group is small
        n_samples=desired_sample_size,  # Resample to desired size
        random_state=42
    )

    resampled_dfs.append(resampled_subset)

# Combine resampled data
data_new = pd.concat(resampled_dfs, axis=0).reset_index(drop=True)

# Verify the new distribution
print(data_new['status'].value_counts())


In [ ]:
x = data_new["stemmed_tokens"]
y = data_new["status"]
y.unique()

In [ ]:
y_encode = to_categorical(y)
y_encode

In [ ]:
x_train, x_temp, y_train, y_temp = train_test_split(x, y_encode, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)
x_train.shape, x_val.shape, x_test.shape, y_train.shape, y_val.shape, y_test.shape

In [ ]:
x.shape, x_temp.shape

In [ ]:
tk = Tokenizer()
tk.fit_on_texts(x_train)
vocab_size = len(tk.index_word) + 1
vocab_size

In [ ]:
import pickle

# Save the tokenizer used during training
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tk, f)

In [ ]:
x_train_number = tk.texts_to_sequences(x_train)
x_val_number = tk.texts_to_sequences(x_val)
x_test_number = tk.texts_to_sequences(x_test)

In [ ]:
max_len = 100
x_train_number = pad_sequences(x_train_number, maxlen=max_len, padding="post")
x_test_number = pad_sequences(x_test_number, maxlen=max_len, padding="post")
x_val_number = pad_sequences(x_val_number, maxlen=max_len, padding="post")
x_train_number.shape, x_val_number.shape, x_test_number.shape

***MULTI CHANNEL CNN MODEL***

In [ ]:
def multi_layer_model(vocab_size, max_length):
    # 1st channel
    inputs1 = Input(shape = (max_length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.7)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    flat1 = Flatten()(pool1)

    # 2nd channel
    inputs2 = Input(shape = (max_length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
    drop2 = Dropout(0.7)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    flat2 = Flatten()(pool2)

    # 3rd channel
    inputs3 = Input(shape = (max_length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
    drop3 = Dropout(0.7)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    flat3 = Flatten()(pool3)


    merged = concatenate([flat1, flat2, flat3])
    outputs = Dense(7, activation='softmax')(merged)
    model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
    model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
filepath="weights_best.keras"
checkpoint = ModelCheckpoint(filepath, monitor= "val_accuracy" , verbose=1, save_best_only=True,mode= "max" )
callbacks_list = [checkpoint]

In [ ]:
model = multi_layer_model(vocab_size=vocab_size, max_length=max_len)
history = model.fit([x_train_number, x_train_number, x_train_number], y_train,
                    validation_data=([x_val_number, x_val_number, x_val_number], y_val),
                    epochs=10,
                    verbose=2,
                    callbacks=callbacks_list)

In [ ]:
model.load_weights("weights_best.keras")
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])

In [ ]:
_, acc = model.evaluate([x_train_number, x_train_number, x_train_number], y_train, verbose=0)
print('Training Accuracy: %f' % (acc*100))
# Validation Accuracy
_, acc = model.evaluate([x_val_number, x_val_number, x_val_number], y_val, verbose=0)
print('Validation Accuracy: %f' % (acc*100))
# Test Accuracy
_, acc = model.evaluate([x_test_number, x_test_number, x_test_number], y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

In [ ]:
pred = np.argmax(model.predict([x_test_number, x_test_number, x_test_number]), axis=1)
pred = Lb.inverse_transform(pred)
pred

In [ ]:
history_df = pd.DataFrame()
for key in history.history.keys():
    history_df[key] = history.history[key]
history_df

In [ ]:
fig = px.line(
    history_df,
    x=history_df.index,
    y=['loss', 'val_loss'],
    title='Model Loss',
    color_discrete_sequence=['green', 'orange']
)
fig.update_xaxes(title_text='Epochs')
fig.update_yaxes(title_text='Loss')
fig.show()


In [ ]:
fig = px.line(
    history_df,
    x=history_df.index,
    y=['accuracy', 'val_accuracy'],
    title='Model Accuracy',
    color_discrete_sequence=['green', 'orange']
)
fig.update_xaxes(title_text='Epochs')
fig.update_yaxes(title_text='Accuracy')
fig.show()


***BERT TRANSFORMER WITH XGBOOST***

In [ ]:
tf.config.run_functions_eagerly(True)
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
print(f"Text input shape: {text_input.shape}, Text input dtype: {text_input.dtype}")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
embedding_model = tf.keras.Model(inputs=[text_input], outputs=[outputs['pooled_output']])

In [ ]:
train_embeddings = embedding_model.predict(x_train)
val_embeddings = embedding_model.predict(x_val)
test_embeddings = embedding_model.predict(x_test)

In [ ]:
# Define the model with parameters
xgb = XGBClassifier(alpha=0.5, lambda_=1.0, learning_rate=0.05, n_estimators=500, early_stopping_rounds=10,eval_metric="logloss")

# Train the model and validate on validation set
xgb.fit(train_embeddings, y_train, eval_set=[(train_embeddings,y_train),(val_embeddings,y_val)], verbose=True)

# Get predictions
y_train_pred = xgb.predict(train_embeddings)
y_val_pred = xgb.predict(val_embeddings)
y_test_pred = xgb.predict(test_embeddings)

# Compute accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
val_accuracy = accuracy_score(y_val, y_val_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# Compute log loss
y_train_prob = xgb.predict_proba(train_embeddings)
y_val_prob = xgb.predict_proba(val_embeddings)
y_test_prob = xgb.predict_proba(test_embeddings)

train_loss = log_loss(y_train, y_train_prob)
val_loss = log_loss(y_val, y_val_prob)
test_loss = log_loss(y_test, y_test_prob)

# Display the results
print(f"Training Accuracy: {train_accuracy:.4f}, Training Loss: {train_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}, Test Loss: {test_loss:.4f}")


In [ ]:
# Save the trained XGBoost model
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(xgb, f)

# Save the label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(Lb, f)

In [ ]:
evals_result = xgb.evals_result()
# Extract the losses for training and validation datasets
training_loss = evals_result['validation_0']['logloss']  # Training loss
validation_loss = evals_result['validation_1']['logloss']  # Validation loss

# Manually calculate accuracy per iteration for training and validation sets
training_accuracy = []
validation_accuracy = []
epochs_to_evaluate = range(0, len(training_loss), 50)  # Sample every 50 epochs

for epoch in epochs_to_evaluate:
    # Predict probabilities at the current stage
    y_train_pred_epoch = xgb.predict(train_embeddings, iteration_range=(0, epoch + 1))
    y_val_pred_epoch = xgb.predict(val_embeddings, iteration_range=(0, epoch + 1))

    # Calculate accuracy for the current epoch
    train_acc = accuracy_score(y_train, y_train_pred_epoch)
    val_acc = accuracy_score(y_val, y_val_pred_epoch)

    training_accuracy.append(train_acc)
    validation_accuracy.append(val_acc)

# Plot Loss Graph
plt.figure(figsize=(12, 6))

# Loss Plot
plt.subplot(1, 2, 1)
plt.plot(range(len(training_loss)), training_loss, label="Training Loss", color="blue")
plt.plot(range(len(validation_loss)), validation_loss, label="Validation Loss", color="orange")
plt.xlabel("Epochs")
plt.ylabel("Log Loss")
plt.title("Training and Validation Loss Over Epochs")
plt.legend()
plt.grid(True)

# Accuracy Plot
plt.subplot(1, 2, 2)
plt.plot(epochs_to_evaluate, training_accuracy, label="Training Accuracy", color="green")
plt.plot(epochs_to_evaluate, validation_accuracy, label="Validation Accuracy", color="red")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training and Validation Accuracy Over Epochs")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


***LSTM Model***

In [ ]:
from tensorflow.keras.regularizers import l2
num_classes = y_train.shape[1]
# LSTM model with L2 regularization
model_b = Sequential([
    tf.keras.Input(shape=(max_len,)),
    Embedding(input_dim=vocab_size, output_dim=128),
    LSTM(units=32),
    Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))  # L2 regularization
])

# Compile the model
model_b.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Early stopping
# early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, verbose=1, restore_best_weights=True)

In [ ]:
# Fit the model
history_b = model_b.fit(x_train_number, y_train,
                        epochs=20,
                        batch_size=32,
                        validation_data=(x_val_number, y_val))

In [ ]:
# Evaluate model_b (LSTM with L2 Regularization)
train_acc_b = model_b.evaluate(x_train_number, y_train, verbose=0)[1]
val_acc_b = model_b.evaluate(x_val_number, y_val, verbose=0)[1]
test_acc_b = model_b.evaluate(x_test_number, y_test, verbose=0)[1]

print("Model B (LSTM with L2 Regularization):")
print(f"  Training Accuracy: {train_acc_b*100:.2f}%")
print(f"  Validation Accuracy: {val_acc_b*100:.2f}%")
print(f"  Test Accuracy: {test_acc_b*100:.2f}%\n")

In [ ]:
# Function to display metrics
def display_metrics(history):
    n = len(history.history['loss'])

    fig = plt.figure(figsize=(12, 6))
    ax = fig.add_subplot(1, 2, 1)
    ax.plot(range(n), history.history['loss'], 'r', label='Training Loss')
    ax.plot(range(n), history.history['val_loss'], 'b', label='Validation Loss')
    ax.legend()
    ax.set_title('Loss over epochs')

    ax = fig.add_subplot(1, 2, 2)
    ax.plot(range(n), history.history['accuracy'], 'r', label='Training Accuracy')
    ax.plot(range(n), history.history['val_accuracy'], 'b', label='Validation Accuracy')
    ax.legend(loc='lower right')
    ax.set_title('Accuracy over epochs')

display_metrics(history_b)

***User Input text Class Prediction***

In [ ]:
from ipywidgets import widgets, VBox, Label, Button, Dropdown
from IPython.display import display
import numpy as np
import pickle
import tensorflow_hub as hub
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load resources
# Tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Stopwords and stemmer
stopwords_list = stopwords.words('english')
stemmer = PorterStemmer()

# XGBoost model
with open('xgb_model.pkl', 'rb') as f:
    xgb_model = pickle.load(f)

# BERT layers
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

# BERT embedding model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
embedding_model = tf.keras.Model(inputs=[text_input], outputs=[outputs['pooled_output']])

# Function: Preprocess input text
def preprocess_text(text):
    tokens = text.split()
    stemmed = [stemmer.stem(word) for word in tokens if word.lower() not in stopwords_list]
    return ' '.join(stemmed)

# Function: Generate BERT embeddings
def get_embeddings(text):
    return embedding_model.predict([text])

# IPyWidgets components
input_box = widgets.Text(
    description="Text:",
    placeholder="Enter a sentence",
    style={"description_width": "initial"},
    layout={"width": "600px"}
)

model_selector = Dropdown(
    options=["LSTM", "CNN", "BERT"],
    description="Model:",
    style={"description_width": "initial"},
    layout={"width": "300px"}
)

output_label = widgets.Label(value="")
predict_button = Button(description="Predict", button_style="success")

# Define the prediction logic
def on_predict_clicked(b):
    selected_model = model_selector.value
    input_text = input_box.value.strip()

    if not input_text:
        output_label.value = "Please enter a valid text."
        return

    # Preprocess input text
    processed_text = preprocess_text(input_text)

    if selected_model == "LSTM":
        # LSTM Model: Single-channel input
        input_sequence = tokenizer.texts_to_sequences([processed_text])
        input_padded = pad_sequences(input_sequence, maxlen=max_len, padding='post')
        input_multi_channel = [input_padded]
        prediction = model_b.predict(input_multi_channel)
        predicted_index = np.argmax(prediction, axis=1)

    elif selected_model == "CNN":
        # CNN Model: Multi-channel input
        input_sequence = tokenizer.texts_to_sequences([processed_text])
        input_padded = pad_sequences(input_sequence, maxlen=max_len, padding='post')
        input_multi_channel = [input_padded, input_padded, input_padded]
        prediction = model.predict(input_multi_channel)  # Replace 'model' with the CNN model variable
        predicted_index = np.argmax(prediction, axis=1)

    elif selected_model == "BERT":
        # BERT Model: Generate embeddings and predict with XGBoost
        bert_embedding = get_embeddings(input_text)
        prediction = xgb_model.predict_proba(bert_embedding)
        predicted_index = np.argmax(prediction, axis=1)

    # Map index to category name
    predicted_category = Lb.inverse_transform(predicted_index)  # Replace 'Lb' with the LabelEncoder object
    output_label.value = f"Predicted Category: {predicted_category[0]}"

# Reset output when the user types new text or changes the model
def on_input_change(change):
    output_label.value = ""  # Clear the output

# Attach event handlers
predict_button.on_click(on_predict_clicked)
input_box.observe(on_input_change, names="value")
model_selector.observe(on_input_change, names="value")

# Display the interface
display(VBox([
    Label("Mental Health Text Classification"),
    model_selector,
    input_box,
    predict_button,
    output_label
]))


***Cross Validation***

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# Assume your data is loaded and preprocessed as x_data and y_data
x_data = x_train_number  # Your input data
y_data = y_train  # Your labels
print(f"x_data size: {len(x_data)}, y_data size: {len(y_data)}")

k = 5  # Number of folds for cross-validation
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Lists to store the performance metrics
cnn_accuracies = []
lstm_accuracies = []

fold_number = 1
# Loop through each fold
for train_index, val_index in kf.split(x_data):
    print(f"Running Fold {fold_number}...")
    # Split the data into training and validation sets for this fold
    x_train, x_val = x_data[train_index], x_data[val_index]
    y_train, y_val = y_data[train_index], y_data[val_index]

    # ----- Multi-Channel CNN Model -----
    cnn_model = multi_layer_model(vocab_size=vocab_size, max_length=max_len)  # Adjust with your parameters
    cnn_model.fit([x_train, x_train, x_train], y_train, epochs=5, verbose=0)
    _, cnn_acc = cnn_model.evaluate([x_val, x_val, x_val], y_val, verbose=0)
    cnn_accuracies.append(cnn_acc)
    print(f"Fold {fold_number} - CNN Accuracy: {cnn_acc * 100:.2f}%")

    # ----- LSTM Model -----
    lstm_model = model_b

    # Compile the model
    lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='accuracy', patience=3, verbose=1, restore_best_weights=True)

    lstm_model.fit(x_train, y_train, epochs=10, verbose=0, callbacks=[early_stopping])
    _, lstm_acc = lstm_model.evaluate(x_val, y_val, verbose=0)
    lstm_accuracies.append(lstm_acc)
    print(f"Fold {fold_number} - LSTM Accuracy: {lstm_acc * 100:.2f}%")

    fold_number += 1

# Calculate average performance metrics
avg_cnn_accuracy = np.mean(cnn_accuracies)
avg_lstm_accuracy = np.mean(lstm_accuracies)

print(f"Average CNN Accuracy: {avg_cnn_accuracy * 100:.2f}%")
print(f"Average LSTM Accuracy: {avg_lstm_accuracy * 100:.2f}%")


***Hypothesis Testing***

In [ ]:
from scipy.stats import ttest_rel

# Perform a paired t-test
t_stat, p_value = ttest_rel(cnn_accuracies, lstm_accuracies)

print("\nHypothesis Testing Results:")
print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis: There is a significant difference between the models.")
else:
    print("Fail to reject the null hypothesis: There is no significant difference between the models.")
